# Трейдерский нейро-помощник

В данном блокноте реализован трейдерский нейро-помощник на основе большой языковой модели GigaChat3 с функционалом управления сделками через function calls. Система объединяет RAG-архитектуру для работы с базой знаний о сделках трейдера и возможность выполнения действий через вызовы функций.


Основная идея данной работы, создать систему, которая будет работать не только как продвинутая вопросно-ответная система, но и решает задачи взаимодействия с помощью естественного языка с операциями на финансовом рынке (этот момент еще дорабатывать, но базовая демонстрация имеется)

**Вводные данные:**

1. Нейро-помощник выполняет роль ассистента для трейдера и управления сделками;
2. База знаний представляет из себя хранилище графов, информация извлекается из текстовых файлов со сделками;
3. Фреймворк LlamaIndex;
4. Модель ai-sage/GigaChat3-10B-A1.8B-bf16 (российская контурная LLM);
5. Функционал function calls для открытия и закрытия сделок;
6. Модерация через OpenAI API;
7. Трассировка запросов и работы модели с помощью Phoenix (Arize Phoenix) для мониторинга и отладки.
8. Улучшение качества ответов и снижение уровня галлюцинаций с помощью метода ресортировки контента


**Задачи нейро-помощника:**

1. Обращаться к базе знаний для получения информации о сделках трейдера.
2. Отвечать точно, основываясь только на данных из источника.
3. Если информация отсутствует, честно говорить, что ответ неизвестен (удалось добиться частично, но это во многом зависит от самой модели).
4. Избегать излишних эмоций, фраз и догадок.
5. Понимать запросы пользователя и выполнять действия через function calls:
   - Открытие новых сделок
   - Закрытие существующих сделок
   - Получение статистики по сделкам
   - Системная функция, которая предположительно будет доступна только пользователям с расширенными правами при реальной работе
   - Возможность расширения и добавления новых функций
6. Автоматически добавлять статистику сделок в контекст для более точных ответов.

По последнему пункту возможно множество расширений, например, в случае многопользовательской модели, когда модель будут использовать несколько человек в одном сервисе, то в системную информацию можно добавлять дополнительные данные по конкретному пользователю, чтобы улучшить поиск в общей базе знаний для всех трейдеров, либо использовать разделенную логику хранения и поиска, но над этим я пока не особо думал, так что не знаю как будет лучше.


## Подготовка


In [37]:
# Установка всех зависимостей одной командой
# !pip install \
#   llama-index-core \
#   "arize-phoenix[evals,llama-index]" \
#   gcsfs \
#   nest-asyncio \
#   "openinference-instrumentation-llama-index>=2.0.0" \
#   llama-index \
#   ipython \
#   langchain \
#   pypdf \
#   langchain_community \
#   llama-index-llms-huggingface \
#   llama-index-embeddings-huggingface \
#   llama-index-embeddings-langchain \
#   langchain-huggingface \
#   sentencepiece \
#   accelerate \
#   bitsandbytes \
#   peft \
#   llama-index-readers-file \
#   ipykernel \
#   llama-index-postprocessor-colbert-rerank \
#   openai \
#   tf-keras

In [38]:
# Путь к директории с документами
source_path = 'data/deals'
deals_registry_path = 'data/deals/deals_registry.txt'


In [39]:
from huggingface_hub import login
import os
from pathlib import Path

HF_TOKEN = os.getenv("HF_TOKEN")

if not HF_TOKEN:
    # Try to read token from .env file in current or parent directories
    from dotenv import load_dotenv, find_dotenv
    dotenv_path = find_dotenv()
    if dotenv_path:
        load_dotenv(dotenv_path)
        HF_TOKEN = os.getenv("HF_TOKEN")
    if not HF_TOKEN:
        raise ValueError("HF_TOKEN environment variable is not set, and was also not found in your .env file. Please set it in your .env file or export it.")

login(HF_TOKEN, add_to_git_credential=True)


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
2025-11-30 17:57:47,806 - WARNING - Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


## Загрузка модели GigaChat3


In [63]:
# Импорт необходимых библиотек
from llama_index.core import SimpleDirectoryReader
from llama_index.core import KnowledgeGraphIndex
from llama_index.core import Settings
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core import StorageContext
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, BitsAndBytesConfig
import torch
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.core.prompts import PromptTemplate
from llama_index.core.postprocessor import LongContextReorder
import getpass
import os
from datetime import datetime
import re


Все работы производил на собственной машине, для запуска в коллабе будет необходима расширенная, платная версия

In [41]:
# Проверка доступности GPU
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    print(f"GPU name: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    device = "cuda"
else:
    print("WARNING: CUDA not available, using CPU")
    device = "cpu"


PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
GPU count: 1
GPU name: NVIDIA GeForce RTX 3090
GPU memory: 24.00 GB


Использовать я решил российскую контурную модель... Ну как российскую, она так или иначе основана на архитектуре DeepSeek, который в свою очередь является продуктом дистилляции моделей таких компаний как OpenAI, Яндекса и других.

Когда я выбирал модель, я забыл, что возможность вызова функций (funtions call) большими языковыми моделями является как дополнительный параметр, а мне, по всей видимости попалась модель без данной вохможности, хотя в самой архитектуре deepseek, исходя из документации, есть такая возможность, но чтобы я ни делал, как бы я ни старался, мне не удалось заставить сгенерировать модель специальные токены во время вызовов функций вроде:

```
<TOOL>
name_of_functiions
</TOOL>
```

Что-то подобное удалось добиться за счет хорошего следования инструкциям, я напрямую написал в системную информацию о наличии функций и как их вызывать и модель, следуя инстуркциям, может их вызывать, а уже мой обработчик ответов, который я внедрил между ответами модели и запросами пользователя, сможет определенным образом на это реагировать. Костыль, я знаю, но я провозился с простым запуском данной модели несколько дней на своей видеокарте, попутно решая проблемы загрузки весов, совместимости библиотек и прочих ошибок. Я использовал до этого модель от Т-Банка, которая на архитектуре Qwen'а, она совершенно точно способна вызвать функции, но я не стал тратить еще неделю, чтобы поменять модель, так как сроки и дедлайн не дадут мне закончить данную работу вовремя.

Мои попытки я все равно решил оставить в коде, где происходит описание функций для параметра tools или tool, так как я использую спец библиотеку для поиска RAG, то я искал в документации по ней как заставить модель вызывать функции и при этом быть способной искать инфу в базе знаний, но что-то особо ничего не получилось, не знаю в чем причина.

Также пробовал описывать функции для вызова по формату OpenAI, чтобы их потом передать в саму модель, но также не получилось.

In [65]:
# Используем модель GigaChat3
model_name = "ai-sage/GigaChat3-10B-A1.8B-bf16"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Настраиваем токенизатор, чтобы не возвращать token_type_ids
# Это необходимо для избежания ошибок с моделями, которые не поддерживают этот параметр
if hasattr(tokenizer, 'return_token_type_ids'):
    tokenizer.return_token_type_ids = False


In [43]:
# Настройка квантизации через BitsAndBytes (4-bit для максимальной экономии памяти)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# Загрузка модели с квантизацией
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto" if torch.cuda.is_available() else None,
    low_cpu_mem_usage=True,
    trust_remote_code=True
)
model.generation_config = GenerationConfig.from_pretrained(model_name)


2025-11-30 17:57:50,710 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

Some weights of the model checkpoint at ai-sage/GigaChat3-10B-A1.8B-bf16 were not used when initializing DeepseekV3ForCausalLM: ['model.layers.26.eh_proj.weight', 'model.layers.26.embed_tokens.weight', 'model.layers.26.enorm.weight', 'model.layers.26.hnorm.weight', 'model.layers.26.input_layernorm.weight', 'model.layers.26.mlp.experts.0.down_proj.weight', 'model.layers.26.mlp.experts.0.gate_proj.weight', 'model.layers.26.mlp.experts.0.up_proj.weight', 'model.layers.26.mlp.experts.1.down_proj.weight', 'model.layers.26.mlp.experts.1.gate_proj.weight', 'model.layers.26.mlp.experts.1.up_proj.weight', 'model.layers.26.mlp.experts.10.down_proj.weight', 'model.layers.26.mlp.experts.10.gate_proj.weight', 'model.layers.26.mlp.experts.10.up_proj.weight', 'model.layers.26.mlp.experts.11.down_proj.weight', 'model.layers.26.mlp.experts.11.gate_proj.weight', 'model.layers.26.mlp.experts.11.up_proj.weight', 'model.layers.26.mlp.experts.12.down_proj.weight', 'model.layers.26.mlp.experts.12.gate_proj.w

In [44]:
# Функции для преобразования промптов для GigaChat3
# GigaChat3 использует стандартный chat template, поэтому используем apply_chat_template
# Используем глобальную переменную для токенизатора (будет обновлена после создания обертки)
_current_tokenizer = tokenizer

def messages_to_prompt(messages):
    """Преобразует сообщения в формат для GigaChat3"""
    # GigaChat3 использует стандартный chat template
    formatted_messages = []
    for message in messages:
        if message.role == 'system':
            formatted_messages.append({"role": "system", "content": message.content})
        elif message.role == 'user':
            formatted_messages.append({"role": "user", "content": message.content})
        elif message.role == 'assistant' or message.role == 'bot':
            formatted_messages.append({"role": "assistant", "content": message.content})
    
    # Используем chat template токенизатора
    prompt = _current_tokenizer.apply_chat_template(
        formatted_messages,
        add_generation_prompt=True,
        tokenize=False
    )
    return prompt

def completion_to_prompt(completion):
    """Преобразует completion в промпт для GigaChat3"""
    messages = [
        {"role": "user", "content": completion}
    ]
    prompt = _current_tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False
    )
    return prompt


In [45]:
# Создание LLM обертки для GigaChat3
generation_config = GenerationConfig.from_pretrained(model_name)
print(generation_config)


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "pad_token_id": 2,
  "repetition_penalty": 1.1,
  "temperature": 0.7,
  "top_p": 0.3
}



In [46]:
# Определяем max_new_tokens с проверкой на None
max_new_tokens_value = 1024  # Значение по умолчанию
if hasattr(generation_config, 'max_new_tokens') and generation_config.max_new_tokens is not None:
    max_new_tokens_value = generation_config.max_new_tokens

# Создаем обертку для токенизатора, которая удаляет token_type_ids (не используется в GigaChat3)
class TokenizerWrapper:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        # Копируем все атрибуты токенизатора
        for attr in dir(tokenizer):
            if not attr.startswith('_') and attr != 'encode':
                try:
                    setattr(self, attr, getattr(tokenizer, attr))
                except:
                    pass
    
    def __call__(self, *args, **kwargs):
        # Удаляем token_type_ids из результата токенизации
        kwargs['return_token_type_ids'] = False
        result = self.tokenizer(*args, **kwargs)
        if isinstance(result, dict) and 'token_type_ids' in result:
            result.pop('token_type_ids')
        return result
    
    def encode(self, *args, **kwargs):
        kwargs['return_token_type_ids'] = False
        return self.tokenizer.encode(*args, **kwargs)
    
    def decode(self, *args, **kwargs):
        return self.tokenizer.decode(*args, **kwargs)
    
    def apply_chat_template(self, *args, **kwargs):
        return self.tokenizer.apply_chat_template(*args, **kwargs)

# Обертываем токенизатор
wrapped_tokenizer = TokenizerWrapper(tokenizer)

# Обновляем глобальную переменную для функций промптов
_current_tokenizer = wrapped_tokenizer

# Настраиваем токенизатор, чтобы не передавать token_type_ids
tokenizer_kwargs = {'return_token_type_ids': False}

llm = HuggingFaceLLM(
    model=model,
    model_name=model_name,
    tokenizer=wrapped_tokenizer,
    max_new_tokens=max_new_tokens_value,
    model_kwargs={"quantization_config": quantization_config},
    tokenizer_kwargs=tokenizer_kwargs,
    generate_kwargs={
        "temperature": 0.2,
        "top_p": 0.9,
        "top_k": 50,
        "do_sample": True,
        "repetition_penalty": 1.1,
    },
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
)


In [47]:
# Настройка модели встраивания
# Отключаем передачу token_type_ids для избежания ошибок
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(
        model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'}
    )
)

# Настройка глобальных параметров
Settings.llm = llm
Settings.embed_model = embed_model
Settings.chunk_size = 512


2025-11-30 17:58:17,563 - INFO - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2


## Функции для возможного вызова моделью


In [48]:
def get_deals_statistics():
    """Получает статистику по сделкам из реестра"""
    try:
        with open(deals_registry_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Подсчет открытых и закрытых сделок
        open_deals = len(re.findall(r'Тип: Открыта', content))
        closed_deals = len(re.findall(r'Тип: Закрыта', content))
        total_deals = len(re.findall(r'Сделка #', content))
        
        return {
            'open': open_deals,
            'closed': closed_deals,
            'total': total_deals
        }
    except FileNotFoundError:
        return {'open': 0, 'closed': 0, 'total': 0}


In [49]:
def get_next_deal_id():
    """Получает следующий доступный ID сделки"""
    try:
        with open(deals_registry_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Находим все ID сделок
        deal_ids = re.findall(r'Сделка #(\d+)', content)
        if deal_ids:
            max_id = max(int(id) for id in deal_ids)
            return f"{max_id + 1:03d}"
        else:
            return "001"
    except FileNotFoundError:
        return "001"


In [50]:
def open_deal(ticker: str, capital_percent: float, entry_price: float, reason: str = ""):
    """
    Открывает новую сделку и записывает её в реестр
    функция для демонстрации работы (заглушена)
    
    Args:
        ticker: Тикер инструмента (например, SBER, GAZP)
        capital_percent: Процент от капитала для сделки
        entry_price: Цена входа
        reason: Основание для сделки (опционально)
    
    Returns:
        dict: Результат операции с ID сделки
    """
    deal_id = get_next_deal_id()
    current_date = datetime.now().strftime("%Y-%m-%d")
    
    deal_entry = f"""Сделка #{deal_id}
    Тикер: {ticker}
    Тип: Открыта
    Дата открытия: {current_date}
    Цена входа: {entry_price}
    Процент от капитала: {capital_percent}%
    Основание: {reason if reason else "Не указано"}
    ---
    """
    
    # Записываем в реестр
    with open(deals_registry_path, 'a', encoding='utf-8') as f:
        f.write(deal_entry)
    
    return {
        "success": True,
        "deal_id": deal_id,
        "message": f"Сделка #{deal_id} успешно открыта. Тикер: {ticker}, Цена входа: {entry_price}, Процент от капитала: {capital_percent}%"
    }


In [51]:
def close_deal(deal_id: str, exit_price: float, exit_reason: str = ""):
    """
    Закрывает существующую сделку в реестре
    функция для демонстрации работы (заглушена)
    
    Args:
        deal_id: ID сделки для закрытия (например, "001")
        exit_price: Цена выхода
        exit_reason: Причина закрытия (опционально)
    
    Returns:
        dict: Результат операции с информацией о прибыли/убытке
    """
    try:
        with open(deals_registry_path, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # Ищем сделку по ID
        pattern = rf'(Сделка #{deal_id}.*?)(?=Сделка #|\Z)'
        match = re.search(pattern, content, re.DOTALL)
        
        if not match:
            return {
                "success": False,
                "message": f"Сделка #{deal_id} не найдена"
            }
        
        deal_text = match.group(1)
        
        # Проверяем, что сделка открыта
        if 'Тип: Закрыта' in deal_text:
            return {
                "success": False,
                "message": f"Сделка #{deal_id} уже закрыта"
            }
        
        # Извлекаем цену входа
        entry_match = re.search(r'Цена входа: ([\d.]+)', deal_text)
        if not entry_match:
            return {
                "success": False,
                "message": f"Не удалось найти цену входа для сделки #{deal_id}"
            }
        
        entry_price = float(entry_match.group(1))
        current_date = datetime.now().strftime("%Y-%m-%d")
        
        # Вычисляем результат
        result_percent = ((exit_price - entry_price) / entry_price) * 100
        result_type = "Прибыль" if result_percent > 0 else "Убыток"
        
        # Обновляем сделку
        updated_deal = deal_text.replace(
            'Тип: Открыта',
            'Тип: Закрыта'
        ).replace(
            '---',
            f'Дата закрытия: {current_date}\nЦена выхода: {exit_price}\nРезультат: {result_type} {abs(result_percent):.2f}%\nПричина закрытия: {exit_reason if exit_reason else "Не указана"}\n---'
        )
        
        # Заменяем в содержимом
        new_content = content.replace(deal_text, updated_deal)
        
        # Записываем обратно
        with open(deals_registry_path, 'w', encoding='utf-8') as f:
            f.write(new_content)
        
        return {
            "success": True,
            "deal_id": deal_id,
            "entry_price": entry_price,
            "exit_price": exit_price,
            "result_percent": result_percent,
            "result_type": result_type,
            "message": f"Сделка #{deal_id} закрыта. {result_type}: {abs(result_percent):.2f}%"
        }
    
    except Exception as e:
        return {
            "success": False,
            "message": f"Ошибка при закрытии сделки: {str(e)}"
        }


In [ ]:
def reindex_knowledge_base():
    """Переиндексирует базу знаний, перечитывая все файлы со сделками"""
    global indexKG, storage_context, query_engine, agent, rag_tool, tools
    
    # Перечитываем документы
    documents = SimpleDirectoryReader(source_path).load_data()
    
    # Создаем новое хранилище
    graph_store = SimpleGraphStore()
    storage_context = StorageContext.from_defaults(graph_store=graph_store)
    
    # Пересоздаем индекс
    indexKG = KnowledgeGraphIndex.from_documents(
        documents=documents,
        max_triplets_per_chunk=10,
        show_progress=True,
        include_embeddings=True,
        storage_context=storage_context
    )
    
    # Сохраняем индекс
    storage_context.persist(persist_dir=storage_path)
    
    # Пересоздаем query engine
    query_engine = indexKG.as_query_engine(include_text=True, verbose=True)
    
    # Пересоздаем RAG инструмент
    rag_tool = QueryEngineTool.from_defaults(
        query_engine=query_engine,
        description="Используй этот инструмент для поиска информации о сделках трейдера в базе знаний."
    )
    
    # Обновляем список инструментов
    tools = [rag_tool, open_deal_tool, close_deal_tool, get_statistics_tool]
    
    # Пересоздаем агента
    agent = ReActAgent(
        tools=tools,
        llm=llm,
        verbose=True,
        system_prompt=create_system_prompt_with_stats()
    )
    
    return "База знаний успешно переиндексирована"


## Создание базы знаний


База знаний представляет из себя реестр сделок, пока только единый реестр для всех трейдеров. Так как я тут пока один, делаю только себя одного, но попутно думаю о многопользовательском режиме, но я все больше склоняюсь к тому, чтобы использовать единый реестр, так как переиндексация может требовать значительного времени, причем, это не настолько сильно зависит от объема самой информации, так как использует одни и те же методы обработки информации для больших файлов и для малых.

А в случае использования единого реестра, можно регулярно каждую ночь через крон выполнять переиндексацию, а ретривер настроить таким образом, чтобы он точно знал что искать для конкретного трейдера, так чтобы при общении с единой моделью каждый трейдер получал персонализированный ответ на основе его данных торговли.

Вообще, у меня есть мысль для оперативной информации использовать более быстрые способы доставки их в модель. Так я реализовал динамический системный промпт для модели, в которую будет записано текущее количество сделок трейдера, открытые, закрытые и всего. Это решает две проблемы: потенционально снижает уровень галлюцинаций модели при генерации ответов, а также не требует постоянного обращения к базе знаний, чтобы получить какую-либо оперативную информацию (не графовую) по данному трейдеру.

Вообще, я выбрал именно графы знаний (Knowledge Graph, KG), так как посчитал, что данный метод идеально подойдет для связанности информации для моего сервиса для трейдеров, например, я делал данную работу под возможность будущего расширения своего другого сервиса по электронному дневнику сделок и уже там у меня возникла мысль объеденить сделки в одну, так у нас сделки обычно исходят из других сделок, например, трейдер совершил первую сделку открытия (лонг или шорт), а затем идут связанные с этой первой сделки другие сделки, такие как усреднения, частичные закрытия и полные закрытия сделок. Это все можно представить в виде графа знаний для модели, надо только наиболее верно структурировать эти данные, чтобы модель, создавая из них граф знаний смогла безошибочно определить связанность информации в едином реестре сделок трейдеров

В идеале я представляю себе это так:




 ```mermaid
 graph TD
     Trader["Трейдер"]
     Trader --> Deals["Сделки"]
     Deals --> MainDeal1["Главная сделка #1 (Открытие позиции)"]
     MainDeal1 --> HelperAvg1["Усреднение"]
     MainDeal1 --> HelperExitPart1["Частичное закрытие"]
     MainDeal1 --> Closing1["Полное закрытие"]
     Deals --> MainDeal2["Главная сделка #2 (Открытие позиции)"]
     MainDeal2 --> HelperAvg2["Усреднение"]
     MainDeal2 --> Closing2["Полное закрытие"]
 ```

 ---
 В этой схеме:
 - Каждый трейдер связан со списком своих сделок.
 - Главная сделка — это сделка открытия позиции (лонг/шорт).
 - Вспомогательные сделки (узлы второго порядка) — это усреднения, частичные закрытия и прочие операции, связанные с основной.
 - Каждая главная сделка заканчивается операцией полного закрытия.
 - Все сделки связаны между собой в рамках единого реестра, что позволяет быстро находить нужную информацию и визуализировать цепочку действий трейдера.

Но пока что же я использую сингл файлы, которые заточены под одного трейдера, это просто как идея для дальнейшего улучшения

In [53]:
# Загрузка документов со сделками
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(source_path).load_data()

# Проверяем загруженные документы
for i, doc in enumerate(documents):
    print(f"Документ {i+1}:\n\n{doc.get_content()[:256]}\n\n")


Документ 1:

Сделка #001
Тикер: SBER
Тип: Открыта
Дата открытия: 2024-01-15
Цена входа: 250.50
Процент от капитала: 5%
Основание: Пробой уровня сопротивления на дневном графике
---
Сделка #002
Тикер: GAZP
Тип: Закрыта
Дата открытия: 2024-01-10
Дата закрытия: 2024-01-20


Документ 2:

История сделок трейдера

Сделка #001
Тикер: SBER
Тип: Открыта
Дата открытия: 2024-01-15
Цена входа: 250.50
Процент от капитала: 5%
Основание: Пробой уровня сопротивления на дневном графике. Объемы выше среднего, формируется восходящий тренд.

Сделка #002
Т




In [54]:
# Создаем графовое хранилище
graph_store = SimpleGraphStore()
storage_context = StorageContext.from_defaults(graph_store=graph_store)


In [55]:
# Построение KnowledgeGraphIndex
# Подавляем предупреждение о неиспользуемом параметре token_type_ids
import warnings
warnings.filterwarnings("ignore", message=".*token_type_ids.*")

# Импорт функции для загрузки индекса
from llama_index.core import load_index_from_storage

# путь к индексу
storage_path = "/home/sigma/projects/neuro-assistant-trader-system/data/index/"

# если индекса нет, то создаем
# иначе загружаем готовый индекс
if not os.path.exists(storage_path) or not os.path.exists(os.path.join(storage_path, "default__vector_store.json")):
    print("Создание нового индекса...")
    indexKG = KnowledgeGraphIndex.from_documents(
        documents=documents,
        max_triplets_per_chunk=10,
        show_progress=True,
        include_embeddings=True,
        storage_context=storage_context
    )
    # Сохраняем индекс после создания
    storage_context.persist(persist_dir=storage_path)
else:
    print("Загрузка существующего индекса...")
    # Загружаем storage_context из сохраненной директории
    storage_context = StorageContext.from_defaults(persist_dir=storage_path)
    # Загружаем индекс из storage_context
    indexKG = load_index_from_storage(storage_context)
    print(indexKG)


2025-11-30 17:58:21,539 - INFO - Loading all indices.


Загрузка существующего индекса...


In [56]:
# Сохранение индекса
# storage_context.persist(persist_dir=storage_path)

## Системный промпт с автоматическим добавлением статистики


В этом системном промпте я и прописал тот самый костыль для возможности вызова функций моделью

In [57]:
def create_system_prompt_with_stats():
    """Создает системный промпт с автоматически добавленной статистикой сделок"""
    stats = get_deals_statistics()
    
    prompt = f"""Ты — нейро-помощник для трейдера и ведения электронного дневника трейдера.
Твоя задача — помогать трейдеру управлять сделками и отвечать на вопросы по его торговой истории.

Текущая статистика сделок:
- Всего сделок: {stats['total']}
- Открытых сделок: {stats['open']}
- Закрытых сделок: {stats['closed']}

Ты обращаешься к Источнику, содержащему данные о всех сделках трейдера, и отвечаешь строго на основе Источника.
Если информация, которую спрашивает пользователь, недоступна в Источнике или ты не уверен в ответе, скажи честно: 'я не знаю'. Не придумывай ответы и не гадай.

КРИТИЧЕСКИ ВАЖНО: Когда пользователь просит что-то сделать или узнать информацию, ТЫ ДОЛЖЕН вызвать соответствующую функцию!

Формат вызова функции - выведи ТОЛЬКО JSON без дополнительного текста:
{{"function": "имя_функции", "arguments": {{"параметр1": "значение1", "параметр2": "значение2"}}}}

Доступные функции и когда их использовать:
1. get_deals_statistics - ВСЕГДА вызывай, когда пользователь спрашивает "сколько сделок", "статистика", "сколько открытых/закрытых" и т.д.
   Формат: {{"function": "get_deals_statistics", "arguments": {{}}}}

2. open_deal - ВСЕГДА вызывай, когда пользователь просит открыть сделку, создать сделку, купить акции и т.д.
   Формат: {{"function": "open_deal", "arguments": {{"ticker": "SBER", "capital_percent": 5, "entry_price": 250.5, "reason": "текст"}}}}
   Пример: "Открой сделку SBER по цене 258" -> {{"function": "open_deal", "arguments": {{"ticker": "SBER", "capital_percent": 2, "entry_price": 258.0, "reason": ""}}}}

3. close_deal - ВСЕГДА вызывай, когда пользователь просит закрыть сделку.
   Формат: {{"function": "close_deal", "arguments": {{"deal_id": "001", "exit_price": 255.0, "exit_reason": "текст"}}}}

4. search_knowledge_base - ВСЕГДА вызывай, когда пользователь спрашивает о сделках, их истории, результатах и т.д.
   Формат: {{"function": "search_knowledge_base", "arguments": {{"query": "вопрос пользователя"}}}}
   Пример: "Какие у меня сделки?" -> {{"function": "search_knowledge_base", "arguments": {{"query": "Какие у меня сделки?"}}}}

5. reindex_knowledge_base - ВСЕГДА вызывай, когда пользователь просит переиндексировать базу знаний.
   Формат: {{"function": "reindex_knowledge_base", "arguments": {{}}}}
   Пример: "Переиндексируй свою базу знаний" -> {{"function": "reindex_knowledge_base", "arguments": {{}}}}

ПРАВИЛА:
- НЕ говори "я не могу" или "функционал недоступен" - ВСЕГДА вызывай функцию, если такие имеются!
- Если пользователь спрашивает о сделках - вызывай search_knowledge_base
- Если пользователь просит открыть сделку - вызывай open_deal
- Если пользователь просит закрыть сделку - вызывай close_deal
- Если пользователь спрашивает статистику - вызывай get_deals_statistics
- Выводи ТОЛЬКО JSON, без дополнительного текста перед или после!

Отвечай четко и по существу, без эмоций. Твоя цель — предоставить точную информацию и помочь трейдеру эффективно управлять сделками."""
    
    return prompt


## Интеграция Function Calls


Мои неудачные попытки заставить модель генерировать специальные токены для вызовов функций

In [ ]:
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.agent import ReActAgent

# Создаем query engine для RAG
query_engine = indexKG.as_query_engine(include_text=True, verbose=True)

# Создаем инструмент для работы с базой знаний через RAG
rag_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    description="Используй этот инструмент для поиска информации о сделках трейдера в базе знаний. Задавай вопросы о конкретных сделках, их истории, результатах и т.д."
)

# Создаем инструменты для function calls
open_deal_tool = FunctionTool.from_defaults(
    fn=open_deal,
    name="open_deal",
    description="Открывает новую торговую сделку. Параметры: ticker (тикер инструмента, строка), capital_percent (процент от капитала, число), entry_price (цена входа, число), reason (основание для сделки, строка, опционально)"
)

close_deal_tool = FunctionTool.from_defaults(
    fn=close_deal,
    name="close_deal",
    description="Закрывает существующую торговую сделку. Параметры: deal_id (ID сделки, строка, например '001'), exit_price (цена выхода, число), exit_reason (причина закрытия, строка, опционально)"
)

get_statistics_tool = FunctionTool.from_defaults(
    fn=get_deals_statistics,
    name="get_deals_statistics",
    description="Получает статистику по всем сделкам: количество открытых, закрытых и всего сделок. Не требует параметров."
)

reindex_knowledge_base_tool = FunctionTool.from_defaults(
    fn=reindex_knowledge_base,
    name="reindex_knowledge_base",
    description="Переиндексирует базу знаний. Используй для обновления информации в базе знаний. Не требует параметров."
)

# Список всех инструментов (RAG + function calls)
tools = [rag_tool, open_deal_tool, close_deal_tool, get_statistics_tool, reindex_knowledge_base_tool]


## Трассировщик запросов Phoenix

С трассировщиком тоже хапанул проблем, пришлось исправлять на лету, чтобы не затягивать работу

In [61]:
# Настройка трассировки Phoenix
import os
import warnings
import logging
import sqlean

# Патч sqlean
if not hasattr(sqlean, 'extensions'):
    from sqlean import extensions
    sqlean.extensions = extensions

if not hasattr(sqlean, 'connect') or not callable(getattr(sqlean, 'connect', None)):
    from sqlean import dbapi2
    if hasattr(dbapi2, 'connect'):
        sqlean.connect = dbapi2.connect

# Импорт Phoenix
import phoenix as px
from phoenix.server.app import PhoenixMigrationError

# Патч percentile для SQLite - улучшенная версия
from sqlalchemy import event
from sqlalchemy.engine import Engine
import sqlalchemy.dialects.sqlite.base as sqlite_base
import sqlalchemy.dialects.sqlite.aiosqlite as aiosqlite_module

class PercentileAggregate:
    """Агрегатная функция для вычисления процентилей в SQLite"""
    def __init__(self):
        self.values = []
        self.p = 0.5
    
    def step(self, value, p):
        if value is not None:
            self.values.append(float(value))
        if p is not None:
            self.p = float(p)
    
    def finalize(self):
        if not self.values:
            return None
        sorted_vals = sorted(self.values)
        index = int(len(sorted_vals) * self.p)
        index = min(max(0, index), len(sorted_vals) - 1)
        return sorted_vals[index]

def add_percentile_to_connection(dbapi_conn):
    """Добавляет функцию percentile к соединению SQLite"""
    try:
        # Пытаемся добавить агрегатную функцию
        if hasattr(dbapi_conn, 'create_aggregate'):
            dbapi_conn.create_aggregate("percentile", 2, PercentileAggregate)
        
        # Также добавляем как обычную функцию (на случай, если Phoenix использует её по-другому)
        if hasattr(dbapi_conn, 'create_function'):
            def percentile_func(value, p):
                # Простая реализация для одного значения
                if value is None or p is None:
                    return None
                return float(value) if isinstance(value, (int, float)) else None
            dbapi_conn.create_function("percentile", 2, percentile_func)
    except Exception as e:
        # Игнорируем ошибки при добавлении функции
        pass

# Патч для синхронных соединений SQLite через SQLAlchemy
@event.listens_for(Engine, "connect")
def receive_connect(dbapi_conn, connection_record):
    add_percentile_to_connection(dbapi_conn)

# Патч для aiosqlite (асинхронный SQLite, используется Phoenix)
try:
    import aiosqlite
    
    # Сохраняем оригинальный класс Connection
    original_aiosqlite_connect = aiosqlite.connect
    
    async def patched_aiosqlite_connect(*args, **kwargs):
        """Обертка для aiosqlite.connect с добавлением percentile"""
        conn = await original_aiosqlite_connect(*args, **kwargs)
        # Добавляем функцию при создании соединения
        add_percentile_to_connection(conn)
        return conn
    
    # Патчим функцию connect
    aiosqlite.connect = patched_aiosqlite_connect
    
    # Также патчим on_connect для SQLAlchemy aiosqlite диалекта
    if hasattr(aiosqlite_module, 'SQLiteDialect'):
        original_on_connect = getattr(aiosqlite_module.SQLiteDialect, 'on_connect', None)
        
        def patched_on_connect(dbapi_conn, connection_record):
            add_percentile_to_connection(dbapi_conn)
            if original_on_connect:
                result = original_on_connect(dbapi_conn, connection_record)
                return result
            return None
        
        aiosqlite_module.SQLiteDialect.on_connect = patched_on_connect
        
        # Также патчим при каждом создании соединения через Connection
        original_connection_init = getattr(aiosqlite.Connection, '__init__', None)
        if original_connection_init:
            def patched_connection_init(self, *args, **kwargs):
                result = original_connection_init(self, *args, **kwargs)
                # Добавляем percentile после инициализации
                # Используем колбэк для асинхронного добавления
                if hasattr(self, '_conn') and self._conn:
                    add_percentile_to_connection(self._conn)
                return result
            
            # Применяем патч только если еще не применен
            if not hasattr(aiosqlite.Connection, '_percentile_patched'):
                aiosqlite.Connection.__init__ = patched_connection_init
                aiosqlite.Connection._percentile_patched = True
except ImportError:
    pass

# Альтернативный патч через SQLAlchemy для aiosqlite
if hasattr(aiosqlite_module, 'SQLiteDialect'):
    # Патчим метод, который создает соединения
    original_get_driver_connection = getattr(aiosqlite_module.SQLiteDialect, '_get_driver_connection', None)
    
    if original_get_driver_connection:
        def patched_get_driver_connection(self, connection):
            result = original_get_driver_connection(self, connection)
            # Добавляем percentile после получения соединения
            if hasattr(result, '_conn'):
                add_percentile_to_connection(result._conn)
            elif hasattr(result, 'connection'):
                add_percentile_to_connection(result.connection)
            return result
        
        aiosqlite_module.SQLiteDialect._get_driver_connection = patched_get_driver_connection

# Патч для компиляции SQL запросов с percentile
def visit_percentile_impl(self, func, **kw):
    """Реализация компиляции SQL для функции percentile"""
    col = self.process(func.clauses.clauses[0], **kw)
    prob = self.process(func.clauses.clauses[1], **kw)
    return f"percentile({col}, {prob})"

# Добавляем поддержку компиляции percentile в SQLAlchemy
if hasattr(sqlite_base, 'SQLiteDialect') and not hasattr(sqlite_base.SQLiteDialect, 'visit_percentile'):
    sqlite_base.SQLiteDialect.visit_percentile = visit_percentile_impl

if hasattr(aiosqlite_module, 'SQLiteDialect') and not hasattr(aiosqlite_module.SQLiteDialect, 'visit_percentile'):
    aiosqlite_module.SQLiteDialect.visit_percentile = visit_percentile_impl

# Дополнительный патч через monkey patching для существующих соединений
try:
    import sqlite3
    
    original_sqlite3_connect = sqlite3.connect
    
    def patched_sqlite3_connect(*args, **kwargs):
        conn = original_sqlite3_connect(*args, **kwargs)
        add_percentile_to_connection(conn)
        return conn
    
    sqlite3.connect = patched_sqlite3_connect
except ImportError:
    pass

# Настройка логирования
warnings.filterwarnings("ignore", message=".*percentile.*")
logging.getLogger("phoenix").setLevel(logging.WARNING)
logging.getLogger("strawberry").setLevel(logging.ERROR)

# Запуск Phoenix
os.environ.setdefault("PHOENIX_HOST", "127.0.0.1")
os.environ.setdefault("PHOENIX_PORT", "6006")

try:
    session = px.launch_app()
    print("🌍 Phoenix: http://localhost:6006/")
except PhoenixMigrationError as e:
    print(f"Ошибка миграции: {e}")
    raise
except Exception as e:
    print(f"Ошибка: {e}")
    raise

2025-11-30 18:02:21,731 - WARNING - Existing running Phoenix instance detected! Shutting it down and starting a new instance...
2025-11-30 18:02:21,972 - INFO - Context impl SQLiteImpl.
2025-11-30 18:02:21,973 - INFO - Will assume transactional DDL.


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix
🌍 Phoenix: http://localhost:6006/


In [62]:
# Настройка инструментации LlamaIndex для трассировки
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

# Настраиваем экспорт трассировки в Phoenix
endpoint = "http://127.0.0.1:6006/v1/traces"
tracer_provider = TracerProvider()
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))

# Инструментируем LlamaIndex для автоматической трассировки всех операций
LlamaIndexInstrumentor().instrument(skip_dep_check=True, tracer_provider=tracer_provider)
print("✅ Трассировка LlamaIndex настроена. Все запросы к RAG и LLM будут отслеживаться в Phoenix.")



2025-11-30 18:02:24,757 - WARNING - Attempting to instrument while already instrumented


✅ Трассировка LlamaIndex настроена. Все запросы к RAG и LLM будут отслеживаться в Phoenix.


![Общий вид трассировщика](img/image1.png)

![Вид трассировки конкретного запроса](img/image2.png)

Вот как раз с классом ниже у меня и возникли проблемы с вызовом функций, вроде и параметр отдельный имеется, но модель почему-то не вызывает ни в какую

Мое предположение таково, что модель просто лишена возможности вызывать функции, она обучена выполнять инструкции, но это совсем не значит, что она может генерировать спец токены для вызова функций. Либо я что-то не так делал, пытаясь разобраться.

In [25]:
# Создаем ReAct агента с инструментами (RAG + function calls)
# Используем конструктор напрямую, так как from_tools может быть недоступен в некоторых версиях
agent = ReActAgent(
    tools=tools,
    llm=llm,
    verbose=True,
    system_prompt=create_system_prompt_with_stats()
)


## Обработка запросов


Это громоздкая функция, дорабатывал ее ИИ под все возможные варианты событий, скорее всего, большая часть условий не используется в примерах ниже, но я производил отладку ячеек, но это приводит к частым зависаниям тетрадки и приходится перезапускать полностью среду, чтобы все заработало, поэтому я после нескольких попыток просто забил, в релизе все равно менять модель на ту которая точно умеет вызывать функции (насчет этой я уже не уверен), а там и библиотеку другую использовать, вроде OpenAI, гораздо более приятная и понятная в использовании, особенно, что касается вызовов функций, там я сразу разобрался, а с текущей библиотекой с поиском в графе знаний все сложно.

Данная функция и вызывает функцию, читает ответы модели и запросы пользователя и на основе этого взаимодействует с системой

In [33]:
def process_query_with_rag_and_functions(query):
    """Обрабатывает запрос пользователя с использованием RAG и function calls через прямой вызов модели"""
    global model, tokenizer, query_engine
    
    import json
    import re
    
    # Обновляем системный промпт с актуальной статистикой
    system_prompt = create_system_prompt_with_stats()
    
    # Определяем tools в формате DeepSeek/OpenAI для function calling
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_deals_statistics",
                "description": "Получает статистику по всем сделкам: количество открытых, закрытых и всего сделок. Не требует параметров.",
                "parameters": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                    "additionalProperties": False
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "open_deal",
                "description": "Открывает новую торговую сделку. Параметры: ticker (тикер акции, строка), capital_percent (процент от капитала, число), entry_price (цена входа, число), reason (причина открытия, строка, опционально)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "ticker": {"type": "string", "description": "Тикер акции (например, SBER, GAZP)"},
                        "capital_percent": {"type": "number", "description": "Процент от капитала для сделки", "minimum": 0, "maximum": 100},
                        "entry_price": {"type": "number", "description": "Цена входа в сделку", "minimum": 0},
                        "reason": {"type": "string", "description": "Причина открытия сделки (опционально)"}
                    },
                    "required": ["ticker", "capital_percent", "entry_price"],
                    "additionalProperties": False
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "close_deal",
                "description": "Закрывает существующую торговую сделку. Параметры: deal_id (ID сделки, строка, например '001'), exit_price (цена выхода, число), exit_reason (причина закрытия, строка, опционально)",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "deal_id": {"type": "string", "description": "ID сделки (например, '001', '002')"},
                        "exit_price": {"type": "number", "description": "Цена выхода из сделки", "minimum": 0},
                        "exit_reason": {"type": "string", "description": "Причина закрытия сделки (опционально)"}
                    },
                    "required": ["deal_id", "exit_price"],
                    "additionalProperties": False
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "search_knowledge_base",
                "description": "Ищет информацию о сделках трейдера в базе знаний. Используй для вопросов о конкретных сделках, их истории, результатах и т.д.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {"type": "string", "description": "Вопрос для поиска в базе знаний"}
                    },
                    "required": ["query"],
                    "additionalProperties": False
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "reindex_knowledge_base",
                "description": "Переиндексирует базу знаний. Используй для обновления информации в базе знаний.",
                "parameters": {}
            }
        }
    ]
    
    # Формируем сообщения для модели с few-shot примерами
    # Добавляем примеры вызова функций в системный промпт
    few_shot_examples = """

Примеры правильного вызова функций:

Пользователь: "Сколько у меня сделок?"
Ты должен ответить ТОЛЬКО JSON:
{"function": "get_deals_statistics", "arguments": {}}

Пользователь: "Какие у меня сделки?"
Ты должен ответить ТОЛЬКО JSON:
{"function": "search_knowledge_base", "arguments": {"query": "Какие у меня сделки?"}}

Пользователь: "Открой сделку по тикеру SBER, цена входа 258.00, процент от капитала 2%"
Ты должен ответить ТОЛЬКО JSON:
{"function": "open_deal", "arguments": {"ticker": "SBER", "capital_percent": 2, "entry_price": 258.0, "reason": ""}}

Пользователь: "Закрой сделку 001 по цене 255"
Ты должен ответить ТОЛЬКО JSON:
{"function": "close_deal", "arguments": {"deal_id": "001", "exit_price": 255.0, "exit_reason": ""}}

ВАЖНО: Отвечай ТОЛЬКО JSON, без дополнительного текста!
"""
    
    messages = [
        {"role": "system", "content": system_prompt + few_shot_examples},
        {"role": "user", "content": query}
    ]
    
    max_iterations = 5  # Максимальное количество итераций для function calling
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        
        # Используем сообщения напрямую (описание tools уже в системном промпте)
        messages_with_tools = messages
        
        # Вызываем модель
        try:
            # Используем chat template для форматирования сообщений
            prompt = tokenizer.apply_chat_template(
                messages_with_tools,
                add_generation_prompt=True,
                tokenize=False
            )
            
            # Токенизируем и генерируем ответ
            # Убеждаемся, что token_type_ids не передается
            inputs = tokenizer(prompt, return_tensors="pt", return_token_type_ids=False).to(model.device)
            # Удаляем token_type_ids из inputs, если он там есть
            if 'token_type_ids' in inputs:
                inputs.pop('token_type_ids')
            
            outputs = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs.get('attention_mask', None),
                max_new_tokens=512,
                temperature=0.2,
                top_p=0.9,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id
            )
            
            # Декодируем ответ
            response_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
            
            # Отладочный вывод (можно убрать позже)
            if iteration == 1:
                print(f"[DEBUG] Ответ модели (итерация {iteration}): {response_text[:500]}")
            
            # Парсим ответ модели на предмет вызова функции
            # Модель должна сама решить, вызывать функцию или нет
            # Ищем структурированный JSON в формате: {"function": "имя", "arguments": {...}}
            function_called = None
            function_args = {}
            
            # Сначала пытаемся найти JSON блок, который начинается с {"function"
            # Ищем от начала строки или после любого текста
            json_start_pattern = r'\{[^{}]*"function"[^{}]*'
            
            # Находим начало JSON блока
            start_match = re.search(json_start_pattern, response_text)
            if start_match:
                start_pos = start_match.start()
                # Пытаемся найти полный JSON блок, начиная с этой позиции
                # Используем простой алгоритм подсчета скобок
                brace_count = 0
                json_str = ""
                in_string = False
                escape_next = False
                
                for i, char in enumerate(response_text[start_pos:], start_pos):
                    json_str += char
                    if escape_next:
                        escape_next = False
                        continue
                    if char == '\\':
                        escape_next = True
                        continue
                    if char == '"':
                        in_string = not in_string
                        continue
                    if not in_string:
                        if char == '{':
                            brace_count += 1
                        elif char == '}':
                            brace_count -= 1
                            if brace_count == 0:
                                # Нашли полный JSON блок
                                try:
                                    parsed = json.loads(json_str)
                                    if isinstance(parsed, dict) and "function" in parsed:
                                        func_name = parsed.get("function")
                                        valid_funcs = [tool["function"]["name"] for tool in tools]
                                        if func_name in valid_funcs:
                                            function_called = func_name
                                            function_args = parsed.get("arguments", {})
                                            break
                                except:
                                    pass
                                break
            
            # Если не нашли через подсчет скобок, пробуем более простой подход
            if not function_called:
                # Ищем JSON блок с помощью регулярного выражения (более гибкий)
                json_pattern = r'\{\s*"function"\s*:\s*"([^"]+)"\s*,\s*"arguments"\s*:\s*(\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\})\s*\}'
                json_match = re.search(json_pattern, response_text, re.DOTALL)
                if json_match:
                    try:
                        func_name = json_match.group(1)
                        args_str = json_match.group(2)
                        valid_funcs = [tool["function"]["name"] for tool in tools]
                        if func_name in valid_funcs:
                            function_called = func_name
                            try:
                                function_args = json.loads(args_str)
                            except:
                                function_args = {}
                    except:
                        pass
            
            # Если все еще не нашли, пробуем найти любой валидный JSON в ответе
            if not function_called:
                # Ищем все возможные JSON блоки
                json_candidates = re.findall(r'\{[^{}]*(?:\{[^{}]*\}[^{}]*)*\}', response_text)
                for candidate in json_candidates:
                    try:
                        parsed = json.loads(candidate)
                        if isinstance(parsed, dict) and "function" in parsed:
                            func_name = parsed.get("function")
                            valid_funcs = [tool["function"]["name"] for tool in tools]
                            if func_name in valid_funcs:
                                function_called = func_name
                                function_args = parsed.get("arguments", {})
                                break
                    except:
                        continue
            
            # Отладочный вывод
            if function_called:
                print(f"[DEBUG] Найдена функция: {function_called}, аргументы: {function_args}")
            else:
                print(f"[DEBUG] Функция не найдена в ответе модели")
            
            # Если функция вызвана, выполняем её
            if function_called:
                if function_called == "get_deals_statistics":
                    result = get_deals_statistics()
                    result_text = f"Статистика: всего {result['total']} сделок, открытых {result['open']}, закрытых {result['closed']}"
                elif function_called == "open_deal":
                    result = open_deal(
                        function_args.get("ticker"),
                        function_args.get("capital_percent"),
                        function_args.get("entry_price"),
                        function_args.get("reason", "")
                    )
                    result_text = result.get('message', str(result))
                elif function_called == "close_deal":
                    result = close_deal(
                        function_args.get("deal_id"),
                        function_args.get("exit_price"),
                        function_args.get("exit_reason", "")
                    )
                    result_text = result.get('message', str(result))
                elif function_called == "search_knowledge_base":
                    rag_query = function_args.get("query", query)
                    rag_response = query_engine.query(rag_query)
                    result_text = rag_response.response
                elif function_called == "reindex_knowledge_base":
                    reindex_knowledge_base()
                    result_text = "База знаний переиндексирована"
                else:
                    result_text = "Неизвестная функция"
                
                # Добавляем результат в историю сообщений
                messages.append({"role": "assistant", "content": response_text})
                messages.append({"role": "user", "content": f"Результат выполнения функции {function_called}: {result_text}. Продолжи ответ пользователю."})
                
                # Продолжаем цикл для получения финального ответа
                continue
            else:
                # Нет вызова функции - возвращаем ответ модели
                # Модель сама решила, что функция не нужна
                return response_text.strip()
        
        except Exception as e:
            return f"Ошибка при обработке запроса: {e}"
    
    # Если превышено количество итераций
    return "Превышено максимальное количество итераций. Попробуйте переформулировать запрос."


## Модерация через OpenAI


Модерация опциональная и использует эндпоинт API OpenAI, конечно, такой варинат не подойдет для корпоративной среды, но добавил это сюда чисто для примера, также можно поднять свой эндпоинт формата OpenAI и просто при инициализации экземпляра класса поменять в атрибуте URL-адреса на свой сервер и все будет работать с вашим собственным сервером.

Просто формат тетрадки не подходит для демонстрации работ систем на основе микросервисов, можно конечно, через сабпроцессы запускать микросервисы, но это походит больше на извращения, нежели на здравое решение.

In [27]:
from openai import OpenAI

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    # Try to read token from .env file in current or parent directories
    from dotenv import load_dotenv, find_dotenv
    dotenv_path = find_dotenv()
    if dotenv_path:
        load_dotenv(dotenv_path)
        OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    if not OPENAI_API_KEY:
        import getpass
        OPENAI_API_KEY = getpass.getpass("Введите OpenAI API Key:")
        os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
    else:
        os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
else:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Инициализация клиента OpenAI (для новой версии API)
try:
    client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
    USE_NEW_API = True
except:
    # Fallback на старую версию
    import openai
    openai.api_key = os.getenv("OPENAI_API_KEY")
    USE_NEW_API = False

# Список категорий, которые надо заблокировать
BLOCKED_CATEGORIES = ["hate", "violence", "self-harm", "sexual"]


In [28]:
def moderate_request_gpt35(prompt):
    """Модерация запроса через OpenAI"""
    try:
        if USE_NEW_API:
            response = client.moderations.create(input=prompt)
            results = response.results[0]
        else:
            response = openai.Moderation.create(input=prompt)
            results = response["results"][0]

        # Проверяем только те категории, которые в списке BLOCKED_CATEGORIES
        if USE_NEW_API:
            categories = results.categories
            flagged_categories = [
                category for category in BLOCKED_CATEGORIES
                if getattr(categories, category, False)
            ]
        else:
            flagged_categories = [
                category for category, flagged in results["categories"].items()
                if flagged and category in BLOCKED_CATEGORIES
            ]

        if flagged_categories:
            return False, flagged_categories
        return True, None
    except Exception as e:
        print(f"Ошибка модерации: {e}")
        # В случае ошибки разрешаем запрос
        return True, None


In [ ]:
def process_request(query, moderate=False):
    """Обрабатывает запрос с опциональной модерацией и функционалом RAG + function calls"""
    # Модерация запроса
    if moderate:
        allowed, categories = moderate_request_gpt35(query)
        if not allowed:
            return f"Запрос заблокирован модерацией из-за содержания в категориях: {categories}"
    
    # Обработка запроса
    response = process_query_with_rag_and_functions(query)
    
    if moderate:
        # Модерация ответа
        allowed_response, response_categories = moderate_request_gpt35(str(response))
        if not allowed_response:
            return "Ответ отклонён модерацией"
    
    return response


## Примеры использования


In [ ]:
# Пример 1: Получение статистики
query = "Какие там у меня сделки?"
response = process_request(query)
print("Ответ:")
print(response)


[DEBUG] Ответ модели (итерация 1): {"function": "search_knowledge_base", "arguments": {"query": "Какие у меня сделки?"}}
[DEBUG] Найдена функция: search_knowledge_base, аргументы: {'query': 'Какие у меня сделки?'}


2025-11-30 01:35:52,720 - INFO - > Querying with idx: 64fd15ea-892a-4c47-99a7-5242ff54e094: История сделок трейдера

Сделка #001
Тикер: SBER
Тип: Открыта
Дата открытия: ...
2025-11-30 01:35:52,720 - INFO - > Querying with idx: 07ae3af7-22b6-4eec-8f5e-4e28ac01cdd4: Результат: Прибыль +4.2%
Причина закрытия: Достигнут целевой уровень прибыли....
2025-11-30 01:35:52,721 - INFO - > Querying with idx: 1a04e31c-d81d-4af0-ae15-e13b71784e61: Сделка #001
Тикер: SBER
Тип: Открыта
Дата открытия: 2024-01-15
Цена входа: 25...


Extracted keywords: ['у', "у меня'", 'какие', "'сделки", 'меня']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Deals', 'Is', 'Example_deals')
('Price entry', 'Is', '180.25')
[DEBUG] Функция не найдена в ответе модели
Ответ:
В вашем контексте также есть две открытые сделки:

1. Сделка №001 с тикером SBER — открыта, дата открытия 2024-01-15, цена входа 250.50, процент от капитала 5%, основание пробой уровня сопротивления на дневном графике.
2. Сделка №003 с тикер YNDX — закрытая, дата открытия 2024-01-05, дата закрытия 2024-01-12, цена вход 3200.00, результат убыток -3.1%.


In [ ]:
# Пример 2: Открытие новой сделки
query = "Открой сделку по тикеру SBER, цена входа 258.00, процент от капитала 2%, основание: пробой сопротивления"
response = process_request(query)
print("Ответ:")
print(response)


[DEBUG] Ответ модели (итерация 1): {"function": "open_deal", "arguments": {"ticker": "SBER", "capital_percent": 2, "entry_price": 258.0, "reason": "пробой сопротивления"}}
[DEBUG] Найдена функция: open_deal, аргументы: {'ticker': 'SBER', 'capital_percent': 2, 'entry_price': 258.0, 'reason': 'пробой сопротивления'}
[DEBUG] Функция не найдена в ответе модели
Ответ:
Отлично! У тебя теперь есть новая сделка с тикером SBER по цене входа 258.0, открытая на 2% от капитала. Следующий шаг будет зависеть от твоей стратегии управления рисками и целей торговли.


In [ ]:
# Пример 3: Вопрос по истории сделок
query = "Расскажи о моих закрытых сделках"
response = process_request(query)
print("Ответ:")
print(response)


[DEBUG] Ответ модели (итерация 1): {"function": "search_knowledge_base", "arguments": {"query": "мои закрытые сделки"}}
[DEBUG] Найдена функция: search_knowledge_base, аргументы: {'query': 'мои закрытые сделки'}


2025-11-29 23:47:27,771 - INFO - > Querying with idx: 1a04e31c-d81d-4af0-ae15-e13b71784e61: Сделка #001
Тикер: SBER
Тип: Открыта
Дата открытия: 2024-01-15
Цена входа: 25...


Extracted keywords: ['мои', 'сделки', 'закрытые']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Deal #003', 'Is', 'Closed deal')
('Deal #002', 'Is', 'Closed deal')
[DEBUG] Функция не найдена в ответе модели
Ответ:
Ваши закрытые сделки: #002 и #003.


In [ ]:
# Пример 4: Закрытие сделки
query = "Закрой сделку #005, цена выхода 259.30, причина: достигнут тейк-профит"
response = process_request(query)
print("Ответ:")
print(response)


[DEBUG] Ответ модели (итерация 1): {"function": "close_deal", "arguments": {"deal_id": "005", "exit_price": 259.3, "exit_reason": "достигнут тейк-профит"}}
[DEBUG] Найдена функция: close_deal, аргументы: {'deal_id': '005', 'exit_price': 259.3, 'exit_reason': 'достигнут тейк-профит'}
[DEBUG] Функция не найдена в ответе модели
Ответ:
Отлично! Сделка #005 успешно закрыта с прибылью в размере 0.50%. Это был хороший результат. Какую стратегию вы планируете применить дальше?


In [ ]:
query = "Переиндексируй свою базу знаний"
response = process_request(query)
print("Ответ:")
print(response)

[DEBUG] Ответ модели (итерация 1): {"function": "reindex_knowledge_base", "arguments": {}}
[DEBUG] Найдена функция: reindex_knowledge_base, аргументы: {}


Parsing nodes:   0%|          | 0/2 [00:00<?, ?it/s]

Processing nodes:   0%|          | 0/5 [00:00<?, ?it/s]

Generating embeddings: 0it [00:00, ?it/s]

Generating embeddings:   0%|          | 0/16 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/22 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/3 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/15 [00:00<?, ?it/s]

[DEBUG] Функция не найдена в ответе модели
Ответ:
{"status": "success", "message": "База знаний успешно переиндексирована."}


## Борьба с галлюцинациями

В данной системе вероятность галлюцинаций относительно низка по нескольким причинам:

1. **Ограниченный функционал**: Модель работает в узкой предметной области (управление сделками трейдера) с четко определенным набором функций.

2. **Простая структура базы знаний**: База знаний содержит структурированную информацию о сделках в текстовом формате, что упрощает извлечение и проверку фактов.

3. **Явные инструкции в системном промпте**: Модель явно инструктируется отвечать "я не знаю", если информация отсутствует в источнике.

4. **Дополнительная мера - ресортировка контента**: В качестве дополнительной меры можно использовать `LongContextReorder` для улучшения качества извлечения релевантной информации из базы знаний, снижая вероятность того, что модель будет опираться на нерелевантные фрагменты контекста. Однако основная работа системы выполняется без ресортировки.

**Слабые стороны в борьбе с галлюцинациями**

Несмотря на принятые меры, система имеет следующие ограничения:

1. **Отсутствие явной верификации фактов**: Система не проверяет факты из базы знаний перед их использованием в ответе.

2. **Ограниченная обработка противоречий**: Если в базе знаний есть противоречивая информация, система может выбрать случайный вариант без явного указания на противоречие.

3. **Зависимость от качества индексации**: Качество ответов напрямую зависит от того, насколько хорошо информация извлечена и проиндексирована в KnowledgeGraphIndex.

4. **Отсутствие метаданных о достоверности**: Система не предоставляет информацию о том, насколько уверена модель в своем ответе или насколько релевантны найденные фрагменты.

5. **Потенциальные проблемы с интерполяцией**: Модель может пытаться "дополнить" информацию из базы знаний своими предположениями, особенно при неполных данных.

Для дальнейшего улучшения можно рассмотреть:
- Добавление системы оценки достоверности ответов
- Внедрение явной верификации фактов через дополнительные запросы к базе знаний
- Использование ансамбля моделей для проверки согласованности ответов
- Добавление метаданных о релевантности найденных фрагментов в ответы модели

In [64]:
# Пример использования ресортировки контента как дополнительной меры борьбы с галлюцинациями
# Создаем отдельный query engine с ресортировкой для сравнения

reorder = LongContextReorder()

# Query engine с ресортировкой контента (дополнительная мера)
query_engine_with_reorder = indexKG.as_query_engine(
    include_text=True,
    verbose=True,
    similarity_top_k=10,
    node_postprocessors=[reorder]
)

query = "Какие у меня открытые сделки по тикеру SBER?"

# Сравнение: ответ без ресортировки (основной query_engine)
response_standard = query_engine.query(query)
print("Ответ БЕЗ ресортировки контента:")
print(response_standard.response)
print("\n" + "="*70)

# Ответ с ресортировкой (дополнительная мера)
response_reordered = query_engine_with_reorder.query(query)
print("Ответ С ресортировкой контента:")
print(response_reordered.response)
print("\n" + "="*70)
print("Ресортировка контента как дополнительная мера помогает:")
print("- Правильно расставить приоритеты между релевантными фрагментами")
print("- Снизить влияние нерелевантных фрагментов на ответ")
print("- Улучшить качество ответов при работе с длинным контекстом")
print("- Снизить вероятность галлюцинаций за счет лучшей структуризации контекста")


2025-11-30 18:18:39,781 - INFO - > Querying with idx: 29dbb21d-90ac-43c3-a314-7adad3cd079d: 00
Цена выхода: 3100.00
Процент от капитала: 4%
Основание: Пробой нисходящего...
2025-11-30 18:18:39,782 - INFO - > Querying with idx: 388b3f3f-712c-4f72-b9e5-0dcc0b0aa020: История сделок трейдера

Сделка #001
Тикер: SBER
Тип: Открыта
Дата открытия: ...
2025-11-30 18:18:39,782 - INFO - > Querying with idx: 4ccfd7b3-6699-4ca0-a047-5b821ed4f964: Результат: Прибыль +4.2%
Причина закрытия: Достигнут целевой уровень прибыли....


Extracted keywords: ['deals', 'open deals', 'SBER', 'open', 'ticker']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Сделка #005', 'Тикer', 'Sber')
('Сделка #004', 'Тикер', 'Sber')
Ответ БЕЗ ресортировки контента:
У вас есть две открытые сделки с тикером SBER:

1. Сделка №001 — открыта, дата открытия 2024-01-15, цена входа 250.50, процент от капитала 5%.
2. Сделка №005 — открыта, дата открытия 2024-01-22, цена входа 0.035, процент от капитала 2%.



2025-11-30 18:19:22,241 - INFO - > Querying with idx: 29dbb21d-90ac-43c3-a314-7adad3cd079d: 00
Цена выхода: 3100.00
Процент от капитала: 4%
Основание: Пробой нисходящего...
2025-11-30 18:19:22,241 - INFO - > Querying with idx: 4ccfd7b3-6699-4ca0-a047-5b821ed4f964: Результат: Прибыль +4.2%
Причина закрытия: Достигнут целевой уровень прибыли....
2025-11-30 18:19:22,241 - INFO - > Querying with idx: 388b3f3f-712c-4f72-b9e5-0dcc0b0aa020: История сделок трейдера

Сделка #001
Тикер: SBER
Тип: Открыта
Дата открытия: ...


Extracted keywords: ['deals', 'open deals', 'SBER', 'open', 'ticker']
KG context:
The following are knowledge sequence in max depth 2 in the form of directed graph like:
`subject -[predicate]->, object, <-[predicate_next_hop]-, object_next_hop ...`
('Сделка #005', 'Цена входа', '258.0')
('Сделка #005', 'Цена входа', '0.035')
('Сделка #004', 'Цена входа', '260.0')
('Сделка #004', 'Тип', 'Открыта')
('Сделка #005', 'Тип', 'Открыта')
('Сделка #005', 'Тикer', 'Sber')
('Сделка #004', 'Тикер', 'Sber')
('Сделка #005', 'Тикер', 'Vtbr')
('Price entry', 'Is', '3200.00')
('Price entry', 'Is', '180.25')
Ответ С ресортировкой контента:
У вас есть две открытые сделки по тикеру SBER:

1. Сделка №004 — открыта с ценой входа 260.0
2. Сделка №005 — открыта с ценой входа 258.0

Ресортировка контента как дополнительная мера помогает:
- Правильно расставить приоритеты между релевантными фрагментами
- Снизить влияние нерелевантных фрагментов на ответ
- Улучшить качество ответов при работе с длинным контексто

## Заключение

### Вывод

В данном блокноте реализован трейдерский нейро-помощник, объединяющий:
1. RAG-систему на основе LlamaIndex с KnowledgeGraphIndex для работы с базой знаний о сделках
2. Модель GigaChat3 (российская контурная LLM) с квантизацией для эффективной работы
3. Function calling для выполнения действий (открытие/закрытие сделок)
4. Автоматическое добавление статистики сделок в контекст
5. Модерацию через OpenAI API для безопасности
6. Возможность переиндексирования базы знаний
7. Трассировку запросов и ответов
8. Борьбу с возможными галлюцинациями и выявление слабых сторон

Система готова к использованию в электронном дневнике трейдера и может быть расширена дополнительными функциями по мере необходимости.

Конечно, стоит сказать, что данная работа это больше демонстрация и определение вектора развития конкретного сервиса, данная работа была выполнена с целью поиска наиболее оптимального решения задачи по разработке ассистента для трейдера и системы трейдера, помощи в осуществлении торговых операция и принятий решения, а также возможность автоматической или полу-автоматической торговли.

Само собой многое еще предстоит сделать, но это уже совсем другая история...